In [1]:
import pandas as pd
import numpy as np

In [2]:
oshima = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Dataset_Processing/yunyoung/data/translate.csv', index_col=0)
display(oshima.head())

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살


In [3]:
#pd.options.display.max_rows = 9999
oshima['info'].value_counts().head(10)

火災による死亡            317
死体発見               238
飛び降り自殺             168
自殺                 142
孤高死                101
告知事項あり（不動産情報より）     94
首吊り自殺               77
転落死                 71
刺殺                  64
告知事項あり              60
Name: info, dtype: int64

In [4]:

#TODO : oshima 범죄 카테고리 상위 10개 
#TODO : < 1차적 목표 > 화재, 시체발견, 자살, 투신자살, 추락사, 부동산 고지사항 카테고리 범주 안에서 태깅 목표(완료)
#TODO : < 2차적 목표 > 상위 10개 제외한 예외 경우 새로운 카테고리 만들어서 태깅
#TODO : < 3차적 목표 > 고지사항의 경우 특정 죽음 키워드(사망, 자살)이 포함된 안내사항 인 경우 태깅하고 그렇지 않으면 데이터 삭제

#? : 告知事項あり（不動産情報より）와 告知事項あり는 같은 부동산 고지사항인데 분류된 기준이 뭐일까? -> 부동산 고지사항에 해당하는 부분 좀 더 파보자.
#* 火災による死亡           317 -> 火災 : 화재
#* 死体発見               238 -> 시체 발견 -> 死体 : 시체 
#* 飛び降り自殺             168 -> 높은 곳에서 뛰어 내려 자살(투신자살) -> 自殺 : 자살, 추가 info
#* 自殺                 142 -> 자살 
#* 孤高死                101 -> 고독사
#* 告知事項あり（不動産情報より）     94 -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용, 좀 더 강조된 걸까?
#* 首吊り自殺               77 -> 목 매달아 자살 -> 자살, 추가 info
#* 転落死                 71 -> 낙하사망(추락사)
#* 刺殺                  64 -> 찔러서 죽인 행위 -> 타살
#* 告知事項あり            60 -> -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용 
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살

In [5]:
oshima.shape

(4833, 8)

In [6]:
#* 주소 중복 데이터 확인됨
df_unique = oshima.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\r\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\r\n특별 모집 주택


In [7]:
# categoryList = oshima['info'].value_counts().head(10).index.tolist()
# categoryList

In [8]:
#* 상위 10개에 해당하는 데이터와 아닌 데이터로 구분
# oshimaTop10 = oshima[oshima['info'].isin(categoryList)].reset_index(drop=True)
# oshimaNotTop10 = oshima[~oshima['info'].isin(categoryList)].reset_index(drop=True)
# print(oshimaTop10.shape, oshimaNotTop10.shape)

In [9]:
# oshimaTop10.loc[:, 'crimeType'] = np.nan
# oshimaNotTop10.loc[:, 'crimeType'] = np.nan

In [10]:
main_keyword_mapping = {
    
    '火災': '화재',
    'アパート火事' : '화재',
    '火事' : "화재",
    '火事が発生しました' : '화재',
    '放火' : '화재',
    '爆死' : '화재',
    '焼死' : '화재',
    '死体': '시체',
    '遺体' : '시체',
    'ミイラ' : '시체',
    '入居者死亡を発見' : '시체',
    '死亡しているのを発見' : '시체',
    '腐敗臭' : '시체',
    '屍体発見されました' : '시체',
    '全身硬直腐乱' : '시체',
    '司法解剖事案' : '시체',
    '腐乱' : '시체',
    '白骨化' : '시체',
    '人骨発見' : '시체',
    '自殺': '자살',
    '首吊り' : '자살',
    '飛び降り' : '자살',
    '無理心中' : '자살',
    '心中' : '자살',
    '縊死' : '자살',
    '患者死亡' : '자살',
    'から投身' : '자살',
    '首つり' : '자살',
    '浴室で縊首' : '자살',
    'お風呂場で自' : '자살',
    '首を絞めて死亡' : '자살',
    '首を吊って死んだ' : '자살',
    '社会的死亡状態' : '고독사',
    '社会死状態で発見され' : '고독사',
    '社会死状態で発見' : '고독사',
    '高齢者が衰弱死' : '고독사',
    '孤高死': '고독사',
    '孤独死' : '고독사',
    'て独' : '고독사',
    '餓死' : '고독사',
    '独居' : '고독사',
    '孤独市' : '고독사',
    '一人で亡くなった' : '고독사',
    '部屋で亡くなった' : '고독사',
    '衰弱死' : '고독사',
    '精神的瑕疵' : '부동산하자',
    '心理的瑕' : '부동산하자',
    '心理的瑕疵あり' : '부동산하자',
    '心理的瑕疵' : '부동산하자',
    '心理的瑕疵物件' : '부동산하자',
    '心理的瑕疵' : '부동산하자',
    '告知事項' : '부동산고지',
    '不動産業者より告知物件との報告有' : '부동산고지',
    '告知事项あり' : '부동산고지',
    '告示事項有' : '부동산고지',
    '告示事項あり' : '부동산고지',
    '告知義務情報あり' : '부동산고지',
    '告知有' : '부동산고지',
    '瑕疵物件' : '부동산고지',
    '事故物件' : '주택내사고',
    '住民が死亡' : '주택내사고',
    '事故住宅' : '주택내사고',
    '事故物件' : '주택내사고',
    '患者死亡' : '주택내사고',
    '室内お亡くなり' : '주택내사고',
    '転落死': '추락사',
    'から転落' : '추락사',
    '転落' : '추락사',
    '飛降り' : '추락사',
    '墜落死' : '추락사',
    '薬物により落下事故' : '추락사',
    '刺殺': '타살',
    '絞殺' : '타살',
    '射殺' : '타살',
    '車両事故死' : '타살',
    '傷害致死' : '타살',
    '撲殺' : '타살',
    '殺人' : '타살',
    '殴殺' : '타살',
    '虐待' : '타살',
    '遺棄' : '타살',
    '殺害' : '타살',
    '殺傷事件' : '타살',
    '斬殺' : '타살',
    '窒息させて殺害' : '타살',
    '生き埋めとなり死亡' : '타살',
    '殴って首を絞めて殺した' : '타살',
    '児投げ落とし事件' : '타살',
    'stabbed' : '타살',
    '幼児殺害' : '타살',
    '暴行死' : '타살',
    '息死' : '타살',
    '病死' : '질병사',
    '脳内出血' : '질병사',
    '過労死' : '질병사',
    '自然死' : '질병사',
    '熱中症による死亡' : '질병사',
    '業務上過失致死' : '질병사',
    '患者死亡' : '질병사',
    '倒れていて死亡' : '질병사',
    'コロナ死' : '질병사',
    '心筋梗塞' : '질병사',
    '凍死' : '질병사',
    '酸欠死' : '질병사',
    '心臓発作による死' : '질병사',
    '事故死' : '사고사',
    '傷害のち死亡' : '사고사',
    'エレベータと壁に挟まれ死亡' : '사고사',
    '中毒' : '사고사',
    '死亡事故' : '사고사',
    'トラックの下敷きになり死亡' : '사고사',
    '通行人をひき死亡する' : '사고사',
    '圧死' : '사고사',
    '溺死' : '사고사',
    '水没' : '사고사',
    '水死' : '사고사',
    '溺殺' : '사고사',
    '感電死' : '사고사',
    '単身事故' : '사고사',
    '変死' : '의문사',
    '不審死' : '의문사',
    '突然死' : '의문사',  
    '原因不明の事故' : '의문사',
    '突然死' : '의문사',
    '未解決事件' : '의문사',
    '不自然死' : '의문사',
    '不明' : '의문사',
    '半死状態' : '의문사',
    '死亡' : '의문사',
    '亡くなっている' : '의문사',
    '不詳の死' : '의문사',
    '失踪' : '의문사',

  
    }
#! 부동산 고지 내용 중 사고가 아닌 것도 있음, 추후 빼야함
#! 特別募集住宅 (특별모집주택)의 경우 사건과 무관한 용어

In [11]:
sub_keyword_mapping = {
    '15:00頃 90代男性が浴室で倒れる、家人が発見？。\r\n救急搬送はしたようだ。' : '질병사',
    '新築住宅資材を積んだトラックが誘導員をひき殺した' : '사고사',
    '検査トラックに頭引かれ即死' : '사고사',
    'ンチン景品交換所店員が強殺。' : '타살',
    '東大医局員が点滴をしたまま亡くなってるのが発見される' : '자살',
    '18歳の子がマンションが飛び込みしぼうしました!' : '자살',
    '午前1時55分頃、東京都立川市羽衣町' : '사고사',
    '3階住民が死後５時間後に発見\r\n二階住民より' : '시체',
    '室内死 3階角部屋' : '주택내사고',
    '一階の部屋で事故' : '주택내사고',
    'サンコート205号室で事故' : '주택내사고',
    'メゾン江古田2階、告知事項有り。\r\n不動産から前の住人が亡くなっていると告知。死因' : '주택내사고',
    '13階 死後3ヶ月後異臭で発見される' : '시체',
    'ラウンドワン三宮店の女子トイレで産まれたばかりの胎児が放棄されてた事件がありました。' : '타살',
    '御爺さん老衰し、その後御婆さん転居し転売される' : '질병사',
     '１階女性 社会死状態にて発見' : '고독사',
     "納品のトラックに下敷きになったアルバイトの脳が飛び散っていたらしい\r\n心霊現象があるらしい" : '초자연현상',
     '402号室にて頭痛や耳鳴り、物が勝手に動く、ありえない落ち方をするなど心霊現象あり' : '초자연현상',
     '隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。' : '초자연현상',
     '女性が亡くなられていた' : '의문사',
     '2Fトイレで男性が亡くなる' : '의문사',
     '205号室/5月2日に人が頭から倒れた様な大きな物音がし' : '고독사',
     'ハイツブルースカイ103\r\n知人が安否確認て訪問、70代男性寝室で発見、死因は他殺の可能性ベランダの窓は開いていて、部屋は荒らされていた。' : '타살',
     '最近住人が亡くなった' : '주택내사고',
     '【青酸カリによる死者が出たそうです】' : '타살',
     '学校不審者侵入事件\r\n死者3名' : '의문사',
     '201号室。病死で室内で発見。告示事項あり。' : '질병사',
     '取引関係のある自動車用品製造、卸売、小売会社の社長３人が首をつって死んでいるのが見つかった。' : '자살',
     '201 告知事項有り 本部屋入居者が2人連続して亡くなられているとのこと。' : '의문사'
}


In [12]:
import warnings
warnings.filterwarnings("ignore")

def taggingF(df, keywordMappingDict):
    df.loc[:, 'crimeType'] = np.nan

    for idx in range(df.shape[0]):
        for key, value in keywordMappingDict.items():
            try:
                if key in str(df.loc[idx, 'info']):
                    df.loc[idx, 'crimeType'] = value
                    break
            except KeyError:
                continue
    return df

In [13]:
df_unique2 = taggingF(df_unique, main_keyword_mapping)
df_unique2['crimeType'].isnull().sum()

99

In [14]:
def subtaggingF(df, keywordMappingDict):
    for idx in range(df.shape[0]):
        for key, value in keywordMappingDict.items():
            try:
                if key in str(df.loc[idx, 'info']):
                    df.loc[idx, 'crimeType'] = value
                    break
            except KeyError:
                continue
    return df

In [15]:
df_unique3 = subtaggingF(df_unique2, sub_keyword_mapping)

In [16]:
# 'info' 열이 'https'로 시작하는 행 삭제
#df_unique3 = df_unique3[~df_unique3['info'].str.startswith('https')]

In [17]:
#* info 칼럼이 영어로 시작하는 행 삭제
df_unique3 = df_unique3[~df_unique3['info'].str.match('^[a-zA-Z]')]

In [18]:
df_unique3['crimeType'].isnull().sum()

65

In [19]:
# df_crime_na = df_unique3.loc[df_unique['crimeType'].isna(), ['info', 'translate', 'crimeType']]
# df_crime_na['info'].value_counts().head(50)
#? 사건과 무관한 사례

In [20]:
df_unique3.shape

(3610, 9)

In [21]:
#* 특별 모집 주택은 사건이 아니므로 삭제
drop_index = df_unique3[df_unique3['info'].str.contains('特別募集')].index 
df_unique3.drop(drop_index, inplace=True)
df_unique3.shape

(3582, 9)

In [22]:
#* 기타 info 중 사건이 아닌 행 삭제
drop_index2 = df_unique3[df_unique3['crimeType'].isna()].index
df_unique3.drop(drop_index2, inplace=True)
df_unique3.shape

(3545, 9)

In [23]:
#* 부동산고지는 사건이 아니므로 삭제
drop_index3 = df_unique3[df_unique3['crimeType'] == '부동산고지'].index
df_unique3.drop(drop_index3, inplace=True)
df_unique3.shape

(2918, 9)

In [24]:
df_unique3.reset_index(drop=True, inplace=True)
df_unique3.shape

(2918, 9)

In [25]:
df_unique3['crimeType'].value_counts()

자살       916
시체       478
고독사      326
화재       306
부동산하자    282
타살       198
의문사      137
질병사      112
추락사      104
사고사       39
주택내사고     17
초자연현상      3
Name: crimeType, dtype: int64

In [26]:
# pd.options.display.max_rows = 1000
# pd.options.display.max_colwidth = 1000
# df_unique3[df_unique3['crimeType'] == '사고사']

In [27]:
df_unique3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   2918 non-null   object 
 1   address    2918 non-null   object 
 2   info       2918 non-null   object 
 3   date       2918 non-null   object 
 4   full name  2918 non-null   object 
 5   latitude   2918 non-null   float64
 6   longitude  2918 non-null   float64
 7   translate  2918 non-null   object 
 8   crimeType  2918 non-null   object 
dtypes: float64(2), object(7)
memory usage: 205.3+ KB


In [28]:
df_unique3.to_csv('oshima_tagging_category.csv', index=False)